# Initialize a new Expectation Suite by profiling a batch of your data.
This process helps you avoid writing lots of boilerplate when authoring suites by allowing you to select columns and other factors that you care about and letting a profiler write some candidate expectations for you to adjust.

**Expectation Suite Name**: `sabari_expectation_suite`


In [6]:
import datetime

import pandas as pd

import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.core.batch import BatchRequest
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.exceptions import DataContextError

context = ge.data_context.DataContext()

batch_request = {'datasource_name': 'first_data', 'data_connector_name': 'default_inferred_data_connector_name', 'data_asset_name': 'first_data.csv', 'limit': 1000}

expectation_suite_name = "sabari_expectation_suite"

validator = context.get_validator(
    batch_request=BatchRequest(**batch_request),
    expectation_suite_name=expectation_suite_name
)
column_names = [f'"{column_name}"' for column_name in validator.columns()]
print(f"Columns: {', '.join(column_names)}.")
validator.head(n_rows=5, fetch_all=False)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Columns: "Unnamed: 0", "Daily Time Spent on Site", "Age", "Area Income", "Daily Internet Usage", "Ad Topic Line", "City", "Male", "Country", "Timestamp", "Clicked on Ad".


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Unnamed: 0  Daily Time Spent on Site  Age  Area Income  \
0           0                     80.67   34     58909.36   
1           1                     68.01   25     68357.96   
2           2                     80.94   36     60803.00   
3           3                     78.77   28     63497.62   
4           4                     36.56   29     42838.29   

   Daily Internet Usage                                   Ad Topic Line  \
0                239.76               Seamless impactful info-mediaries   
1                188.32                 Ameliorated actuating workforce   
2                239.94                    Self-enabling local strategy   
3                211.83  Public-key intangible Graphical User Interface   
4                195.89       Team-oriented bi-directional secured line   

            City  Male      Country            Timestamp  Clicked on Ad  
0     Jamesville     0     Portugal  2016-01-01 02:52:10              0  
1     Kaylashire     1  Afghanistan  2016-01-01 03:35:35              0  
2  North Tiffany     0      Bermuda  2016-01-01 05:31:22              0  
3      Gomezport     0         Guam  2016-01-01 08:27:06              0  
4      West Ryan     0       Uganda  2016-01-01 15:14:24              1

# Select columns

Select the columns on which you would like to set expectations and those which you would like to ignore.

Great Expectations will choose which expectations might make sense for a column based on the **data type** and **cardinality** of the data in each selected column.

Simply comment out columns that are important and should be included. You can select multiple lines and use a Jupyter
keyboard shortcut to toggle each line: **Linux/Windows**:
`Ctrl-/`, **macOS**: `Cmd-/`

Other directives are shown (commented out) as examples of the depth of control possible (see documentation for details).


In [7]:
exclude_column_names = [
    "Unnamed: 0",
#     "Daily Time Spent on Site",
#     "Age",
    "Area Income",
    "Daily Internet Usage",
    "Ad Topic Line",
    "City",
    "Male",
    "Country",
    "Timestamp",
    "Clicked on Ad",
]

# Run the OnboardingDataAssistant

The suites generated here are **not meant to be production suites** -- they are **a starting point to build upon**.

**To get to a production-grade suite, you will definitely want to [edit this
suite](https://docs.greatexpectations.io/docs/guides/expectations/create_expectations_overview#editing-a-saved-expectation-suite)
after this initial step gets you started on the path towards what you want.**

This is highly configurable depending on your goals.
You can ignore columns, specify cardinality of categorical columns, configure semantic types for columns, even adjust thresholds and/or different estimator parameters, etc.
You can find more information about OnboardingDataAssistant and other DataAssistant components (please see documentation for the complete set of DataAssistant controls) [how to choose and control the behavior of the DataAssistant tailored to your goals](https://docs.greatexpectations.io/docs/guides/expectations/data_assistants/how_to_create_an_expectation_suite_with_the_onboarding_data_assistant).

Performance considerations:
- Latency: We optimized for an explicit "question/answer" design, which means we issue **lots** of queries. Connection latency will impact performance.
- Data Volume: Small samples of data will often give you a great starting point for understanding the dataset. Consider configuring a sampled asset and profiling a small number of batches.
    

In [8]:
result = context.assistants.onboarding.run(
    batch_request=batch_request,
    exclude_column_names=exclude_column_names,
)
validator.expectation_suite = result.get_expectation_suite(
    expectation_suite_name=expectation_suite_name
)

Generating Expectations:   0%|          | 0/8 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/1 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/0 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/0 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/2 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/2 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/0 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/0 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/1 [00:00<?, ?it/s]

C:\Python310\lib\site-packages\great_expectations\core\util.py:263: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, val in data.iteritems()


# Save & review your new Expectation Suite

Let's save the draft expectation suite as a JSON file in the
`great_expectations/expectations` directory of your project and rebuild the Data
 Docs site to make it easy to review your new suite.

In [9]:
print(validator.get_expectation_suite(discard_failed_expectations=False))
validator.save_expectation_suite(discard_failed_expectations=False)

checkpoint_config = {
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name
        }
    ]
}
checkpoint = SimpleCheckpoint(
    f"{validator.active_batch_definition.data_asset_name}_{expectation_suite_name}",
    context,
    **checkpoint_config
)
checkpoint_result = checkpoint.run()

context.build_data_docs()

validation_result_identifier = checkpoint_result.list_validation_result_identifiers()[0]
context.open_data_docs(resource_identifier=validation_result_identifier)

2022-09-28T14:08:19+0530 - INFO - 	21 expectation(s) included in expectation_suite.
{
  "data_asset_type": null,
  "expectations": [
    {
      "kwargs": {
        "max_value": 610,
        "min_value": 610
      },
      "expectation_type": "expect_table_row_count_to_be_between",
      "meta": {
        "profiler_details": {
          "metric_configuration": {
            "metric_name": "table.row_count",
            "domain_kwargs": {},
            "metric_value_kwargs": null,
            "metric_dependencies": null
          },
          "num_batches": 1
        }
      }
    },
    {
      "kwargs": {
        "column_set": [
          "Daily Internet Usage",
          "Country",
          "Male",
          "Age",
          "Timestamp",
          "Clicked on Ad",
          "City",
          "Area Income",
          "Daily Time Spent on Site",
          "Unnamed: 0",
          "Ad Topic Line"
        ],
        "exact_match": null
      },
      "expectation_type": "expect_table_col

2022-09-28T14:08:19+0530 - INFO - 	21 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/32 [00:00<?, ?it/s]

C:\Python310\lib\site-packages\great_expectations\expectations\metrics\map_metric_provider.py:1428: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return list(domain_values[: result_format["partial_unexpected_count"]])


## Next steps
After you review this initial Expectation Suite in Data Docs you
should edit this suite to make finer grained adjustments to the expectations.
This can be done by running `great_expectations suite edit sabari_expectation_suite`.